In [ ]:
pip install spotipy

In [ ]:
pip install pandas

In [ ]:
pip install time

In [6]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
import pandas as pd

# 設置 Client ID 和 Secret
client_id = "45da58455f7f46d5b30b0b8a117fa452"
client_secret = "7dd2efbeaecd4fd08136e682f7610f48"

# 授權
def get_spotify_client():
    client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    return sp

# 獲取專輯歌曲，並處理速率限制
def get_album_track(sp, album_id):
    
    while True:
        try:
            data = sp.album_tracks(album_id)
            tracks = data.get('items', [])
            album_tracks = []
            for track in tracks:
                track_id = track.get("id")  # 取得歌曲 ID
                track_name = track.get("name")  # 取得歌曲 名稱
                track_ms = track.get("duration_ms") # 取得歌曲 長度
                track_number = track.get("track_number") # 取得歌曲 編號
                album_tracks.append((track_id, track_name, track_ms, track_number))  # 將 專輯 info 添加到列表中
            return album_tracks
        
        except spotipy.exceptions.SpotifyException as e:
            if e.http_status == 401:  # Unauthorized, possibly due to an expired token
                print("Access token expired. Refreshing token...")
                sp = get_spotify_client()  # Get a new Spotify client with refreshed token
            
            elif e.http_status == 429:  # Too Many Requests
                retry_after = int(e.headers.get("Retry-After", 1))
                print(f"Rate limit exceeded. Retrying after {retry_after} seconds...")
                time.sleep(retry_after)
            
            else:
                print(f"Failed to get tracks for album {album_id}. Error: {str(e)}")
                return []
# 主程式
def main():
    sp = get_spotify_client()

    # 讀取 CSV
    artist_album_ids_path = r"C:\Users\T14 Gen 3\Desktop\KKbox_Spotify_va13\artist_album_ids.csv"
    df = pd.read_csv(artist_album_ids_path)

    # 將查詢結果存入 DataFrame
    result_data = []
    
    for index, row in df.iterrows():
        album_id = row["Artist_album_id"]

        try:
            album_track_info = get_album_track(sp, album_id)

            if album_track_info:
                for track_id, track_name, track_ms, track_number in album_track_info:  # 正確解包專輯 info
                    result_data.append({"Artist_album_id": album_id, 
                                        "Track_id": track_id, 
                                        "Track_name": track_name, 
                                        "Track_ms": track_ms, 
                                        "Track_number": track_number})
            else:
                print(f"Album ID {album_id} has no tracks.")
        
        except Exception as e:
            print(f"Error processing album {album_id}: {e}")
    
        # 添加延遲，防止達到速率限制
        time.sleep(1)  # 每次請求後等待 1 秒
            
    result_df = pd.DataFrame(result_data)

    # 保存結果到 CSV
    result_csv_path = r"C:\Users\T14 Gen 3\Desktop\KKbox_Spotify_va13\album_tracks_2.csv"
    result_df.to_csv(result_csv_path, index=False)
    

if __name__ == "__main__":
    main()
